In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import PyPDFLoader
import nltk
from langchain_text_splitters import NLTKTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from IPython.display import Markdown as md
from dotenv import load_dotenv
import os

In [6]:
load_dotenv()  
key = os.getenv("GOOGLE_API_KEY")

In [7]:
chat_model = ChatGoogleGenerativeAI(google_api_key=key, 
                                   model="gemini-1.5-pro-latest")
loader = PyPDFLoader("../data/1neural_network.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'producer': '適用於 Microsoft 365 的 Microsoft® PowerPoint®', 'creator': '適用於 Microsoft 365 的 Microsoft® PowerPoint®', 'creationdate': '2024-02-19T15:15:35+08:00', 'title': '深度學習簡介', 'author': 'fhwang', 'moddate': '2024-02-19T15:15:35+08:00', 'source': '../data/1neural_network.pdf', 'total_pages': 20, 'page': 0, 'page_label': '1'}, page_content='類神經網路基礎\n王豐緒\n銘傳大學資工系'),
 Document(metadata={'producer': '適用於 Microsoft 365 的 Microsoft® PowerPoint®', 'creator': '適用於 Microsoft 365 的 Microsoft® PowerPoint®', 'creationdate': '2024-02-19T15:15:35+08:00', 'title': '深度學習簡介', 'author': 'fhwang', 'moddate': '2024-02-19T15:15:35+08:00', 'source': '../data/1neural_network.pdf', 'total_pages': 20, 'page': 1, 'page_label': '2'}, page_content='學習目標\n• 理解類神經元的基本結構與運作方式\n• 理解何謂Perceptron類神經網路\n• 理解類神經的學習方式\n• 理解類神經的訓練與測試過程\n• 理解矩陣運算與類神經的關聯\n2'),
 Document(metadata={'producer': '適用於 Microsoft 365 的 Microsoft® PowerPoint®', 'creator': '適用於 Microsoft 365 的 Microsoft® PowerPoint®', 'creationd

In [8]:
text_splitter = NLTKTextSplitter(chunk_size=500, chunk_overlap=100)

chunks = text_splitter.split_documents(pages)
print(len(chunks))
print(type(chunks[0]))

20
<class 'langchain_core.documents.base.Document'>


In [9]:
embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=key, model="models/embedding-001")
db = Chroma.from_documents(chunks, embedding_model, persist_directory="../chroma_db_")
db.persist()
db_connection = Chroma(persist_directory="../chroma_db_", embedding_function=embedding_model)

/tmp/ipykernel_27887/164989724.py:3: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()
/tmp/ipykernel_27887/164989724.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db_connection = Chroma(persist_directory="../chroma_db_", embedding_function=embedding_model)


In [10]:
retriever = db_connection.as_retriever(search_kwargs={"k": 5})

print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [11]:
chat_template = ChatPromptTemplate.from_messages([
    SystemMessage(content="""You are a teacher in Scaffolding Instruction education.
                  Given a context and question from user,
                  you should answer based on the given context."""),
    HumanMessagePromptTemplate.from_template("""Answer the question based on the given context.
    Context: {context}
    Question: {question}
    Answer: """)
])

output_parser = StrOutputParser()


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser
)

In [12]:
response = rag_chain.invoke("""Please summarize what is a neural network""")
md(response)

Based on the provided text, a neural network is inspired by biological neurons in the brain.  Artificial neurons, like their biological counterparts, receive inputs (x1, x2,...xm), each weighted (w1, w2,...wm), and a bias (𝜃). These weighted inputs are summed and passed through an activation function (g) to produce an output (o).  The activation function determines whether the neuron "fires" (outputting 1) or not (outputting 0) based on whether the weighted sum of inputs exceeds a threshold.  Learning in artificial neural networks involves adjusting the weights and bias.  The text mentions the McCulloch and Pitts neuron model as an example of an artificial neuron and describes how biological neurons connect through synapses, the strengths of which are modified through learning.  While the text outlines topics related to perceptron neural networks, learning equations, and training/testing phases, it does not provide a comprehensive definition of a neural network beyond the individual neuron model.

In [13]:
response = rag_chain.invoke("""詳細解釋權重""")

md(response)

提供的上下文並沒有詳細解釋權重，只提到初始化權重、訓練階段會更新權重以得到正確答案，以及類神經網路用(W)代表權重。並沒有說明權重如何計算、更新，或其代表的意義。

要理解權重，需要補充以下資訊：

* **權重的意義:**  在類神經網路中，權重代表輸入數據的重要性。每個輸入值都會乘以一個權重，然後加總起來，再經過激活函數處理，最終產生輸出。權重越大，表示對應的輸入值對輸出的影響越大。

* **權重的初始化:**  在開始訓練之前，需要初始化權重。常見的初始化方法包括隨機初始化、使用特定分佈（例如高斯分佈）初始化等。初始化的值會影響訓練的效率和最終結果。

* **權重的更新:**  在訓練過程中，類神經網路會根據預測結果和實際結果之間的差異（損失函數）來調整權重。這個調整過程通常使用反向傳播算法和梯度下降等優化算法。目標是找到一組權重，使得損失函數最小化，也就是讓網路的預測結果盡可能接近實際結果。

總之，權重是類神經網路的核心組成部分，它們決定了網路如何處理輸入數據並產生輸出。透過訓練過程不斷調整權重，網路可以學習到從輸入到輸出的映射關係。